In [1]:
# !huggingface-cli login

In [1]:
from typing import cast
import datasets

dataset = cast(datasets.DatasetDict, datasets.load_dataset("aynumosir/ainu-corpora"))

In [21]:
import json

from collections import defaultdict
from itertools import chain
from typing import cast, TypedDict
from utils.tokenize import tokenize


class Text(TypedDict):
    book: str
    title: str
    url: str
    pronoun: str
    author: str
    dialect: str
    text: str
    translation: str
    label: str



class CorpusItem(TypedDict):
    translation: str
    sentence: str
    words: list[str]
    part_of_speech: list[str]

sentences_by_book = defaultdict(list)

with open("../dictionary/output/wiktionary_ainu_part_of_speech.json", "r") as f:
    part_of_speech = json.load(f)

for text in chain(dataset["test"], dataset["train"]):
    text = cast(Text, text)
    words = tokenize(text["text"])
    sentences_by_book[text["book"]].append(
        {
            "translation": text["translation"],
            "sentence": text["text"],
            "words": words,
            "part_of_speech": [
                part_of_speech.get(w, []) for w in words
            ],
        }
    )

sentences_by_book.keys()

dict_keys(['アイヌ語鵡川方言日本語‐アイヌ語辞典', 'アイヌ語アーカイブ', 'アイヌタイムズ', '平取町アイヌ口承文芸', 'アイヌ語アーカイブ音声資料', 'AA研アイヌ語資料', '浅井タケ昔話全集I,II', '石狩川のアイヌ語（中級）', 'アイヌ語會話字典', 'アイヌ語口承文芸コーパス', '石狩川のアイヌ語（初級）', '十勝のアイヌ語（初級）', 'アイヌ語ラジオ講座テキスト', 'アイヌ民族文化センター研究紀要', 'アイヌ語音声資料', '十勝のアイヌ語（中級）', '沙流のアイヌ語（初級）', 'カラフトのアイヌ語（中級）', '美幌のアイヌ語（初級）', '鍋沢元蔵筆録ノート', 'アコㇿイタㇰ', '千徳太郎治のピウスツキ宛書簡', '沙流のアイヌ語（中級）', 'カムイユカㇻを聞いてアイヌ語を学ぶ', 'ニューエクスプレスプラス アイヌ語', '萱野茂の国会演説', 'アイヌ神謡集', '幌別のアイヌ語（初級）', '幌別のアイヌ語（中級）', 'ちとせのアイヌご（入門）', '千歳のアイヌ語（初級）', '静内のアイヌ語（初級）', '静内のアイヌ語（中級）', 'プラハ宣言', '美幌のアイヌ語（中級）', 'びほろのあいぬご（入門）', 'ほろべつのアイヌご（入門）', 'カラフトのアイヌ語（初級）', '千歳のアイヌ語（中級）', 'さるのアイヌご（入門）', 'しずないのアイヌご（入門）', 'いしかりがわのアイヌご（入門）', 'ウポポイ館内展示', 'ニューエクスプレス・スペシャル 日本語の隣人たち I+II', 'とかちのアイヌご（入門）', 'からふとのアイヌご（入門）', '第27回アイヌ語弁論大会', 'ピウスツキ記念碑'])

In [23]:
sentences = print(sum(len(v) for v in sentences_by_book.values()))

174585


In [24]:
for sentence in (
    sentences_by_book["萱野茂の国会演説"][0:3]
    + sentences_by_book["平取町アイヌ口承文芸"][-3:]
    + sentences_by_book["AA研アイヌ語資料"][55 : 55 + 3]
    + sentences_by_book["沙流のアイヌ語（中級）"][10:13]
):
    print("sentence: ", sentence["sentence"])
    print("translation: ", sentence["translation"])
    print("words: ", sentence["words"])
    print("part of speech: ", sentence["part_of_speech"])
    print("-" * 100)

sentence:  ne hi or ta nispa utar ku=koramkor hi ene oka hi aynumosir un aynu utar ka koeturenno sisam mosir ta oka aynu ka ene ne yakka aynu néno aynuitak ani ukoytak pa wa ratci oka apunno oka kuni kosanniyo wa un=kore yan
translation:  そこで、私が先生方にお願いしたいということは、北海道にいるアイヌたち、それと一緒に、各地にいるアイヌたちがどのようにしたらアイヌ民族らしくアイヌ語で会話を交わし、静かに豊かに暮らしていけるかを先生方に考えてほしいと私は思い、
words:  ['ne', 'hi', 'or', 'ta', 'nispa', 'utar', 'ku=', 'koramkor', 'hi', 'ene', 'oka', 'hi', 'aynumosir', 'un', 'aynu', 'utar', 'ka', 'koeturenno', 'sisam', 'mosir', 'ta', 'oka', 'aynu', 'ka', 'ene', 'ne', 'yakka', 'aynu', 'n', 'no', 'aynuitak', 'ani', 'ukoytak', 'pa', 'wa', 'ratci', 'oka', 'apunno', 'oka', 'kuni', 'kosanniyo', 'wa', 'un=', 'kore', 'yan']
part of speech:  [['pron', 'pronoun', 'verb', 'determiner', 'parti'], ['noun'], ['noun'], ['verb', 'parti'], ['noun'], ['noun'], ['prefix'], [], ['noun'], ['adv'], ['verb', 'pron', 'noun'], ['noun'], [], ['verb', 'parti'], ['noun'], ['noun'], ['parti', 'noun'], [], ['noun'], ['noun'], ['

In [25]:
import pickle

with open("output/annotated_translated_tokenized_corpus_by_book.pkl", "wb") as f:
    pickle.dump(sentences_by_book, f)